In [1]:
import pandas as pd
import numpy as np
import json
import folium

This notebook creates maps for each that show the percentage of agreement for each thematique.

Load the data for the % by commune and for the the theme of each votation.

In [2]:
data = pd.read_pickle("data/data.pkl")
thematique = pd.read_pickle("data/Thématique.pkl")

load the json to create the commune on the map, and create the lsit of the commune in the json. 

In [3]:
switzerland_coord = [46.765213, 8.252444]
town_geo_path = r'data/switzerland_borders/admin_level_8.geojson'
geo_json_data = json.load(open(town_geo_path, encoding="utf8"))
commune = [x['name']  for x in geo_json_data['features']]



We merge the 2 dataframes so that we have for each votation and each commune the theme and the percentage of yes. We only takes 'Thématique','Commune' and 'Oui en %' because it will be the only usefull information for later (Votation is no longer use full once we managed to merge)

We also make sure thatthere is in the dtaframes only commune that are in the json.

In [4]:
data_t = data.merge(thematique , on = 'Votation')[['Thématique','Commune','Oui en %']]

data_t = data_t[data_t['Commune'].isin(commune)]
data_t.head()

,Thématique,Commune,Oui en %
0,Politique sociale,Aeugst am Albis,34.9
1,Politique sociale,Affoltern am Albis,30.3
2,Politique sociale,Bonstetten,34.3
3,Politique sociale,Hausen am Albis,35.5
4,Politique sociale,Hedingen,31.1


We group the data by theme, and for each one we create a map showing how much people voted yes. We then save it into an html.

In [5]:

for theme, data_theme in data_t.groupby('Thématique') :
    data_theme = data_theme.groupby('Commune', as_index  = False).mean()
    map1 = folium.Map(location=switzerland_coord, zoom_start=8)
    map1.choropleth(geo_data = geo_json_data, \
                                    data = data_theme, \
                                    columns = ['Commune', 'Oui en %'], \
                                    key_on = 'feature.name', \
                                    fill_color = 'RdYlGn', \
                                    fill_opacity = 0.7, \
                                    line_opacity = 0.2, \
                                    legend_name = 'yes in % given to the theme ' + theme)
    
    map1.save('data/map_theme/map_'+theme+'.html')